In [1]:
import pandas as pd
import numpy as np

### DataFiltring

In [2]:
# first problem
def big_countries(world:pd.DataFrame) -> pd.DataFrame:
    df = world.copy()
    df = df[(df['area']>= 3000000)|(df['population']>=25000000)]
    return df[['name','population','area']]

#second problem
def find_products(products: pd.DataFrame)-> pd.DataFrame:
    df = products.copy()
    df = df[(df['low_fats'] == 'Y')&(df['recyclable'] == 'Y')]
    return df[['product_id']]

#third problem
def find_customers(customers:pd.DataFrame, orders:pd.DataFrame)->pd.DataFrame:
    df = customers.copy()
    return df[~ df['id'].isin(orders['customerId'])].rename(columns={"name":'Customers'})[['Customers']]

#fourth problem
def article_views(views:pd.DataFrame)->pd.DataFrame:
    df = views.copy()
    return df[df['author_id'] == df['viewer_id']][['author_id']].rename(columns={'author_id':'id'}).\
    drop_duplicates().sort_values(by='id')

### string methods

In [3]:
#first problem
def invalid_tweets(tweets:pd.DataFrame)->pd.DataFrame:
    df = tweets.copy()
    return df[df['content'].str.len() > 15][['tweet_id']]

#second problem
def calculate_special_bouns(employees:pd.DataFrame)-> pd.DataFrame:
    df = employees.copy()
    df['bouns'] = df.apply(lambda row : row['salary'] if (row['employee_id']%2 !=0)and(not row['name'].startswith('M')) else 0 ,axis=1)
    return df[['employee_id','bouns']].sort_values(by='employee_id')
# another solution using assign 
def calculate_special_bonus(employees: pd.DataFrame) -> pd.DataFrame:
    return employees.assign(
        bonus=employees.apply(lambda x: x['salary'] if int(x['employee_id']) % 2 != 0 and not x['name'].startswith('M') else 0, axis=1)
    )[['employee_id', 'bonus']].sort_values(
        by='employee_id',
    )

#third problem
def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    df = users.copy()
    df['name'] = df['name'].str.capitalize()
    return df.sort_values(by='user_id')
#another solution using the assign method
def fix_names(users:pd.DataFrame)-> pd.DataFrame:
    df = users.copy()
    return df.assign(
    name = df['name'].str.capitalize()
    ).sort_values(by='user_id')

#fourth problem
def valid_emails(users: pd.DataFrame) -> pd.DataFrame:
    return users[
        users['mail'].str.match(r'^[a-zA-Z][a-zA-Z\d_.-]*@leetcode\.com$')
    ]

#fifth problem
def find_patients(patients: pd.DataFrame) -> pd.DataFrame:
    return patients[
        patients['conditions'].str.contains(r'(^DIAB1)|( DIAB1)')
    ]

### Data Manipulation

In [4]:
# first problem 
def nth_highest_salary(employee:pd.DataFrame, N:int)->pd.DataFrame:
    df = employee.copy()
    df = df.drop_duplicates(subset='salary')
    if N > len(df) or N<1:
        result = {f' getNthHighestSalary({N})':np.NaN}
        result_df = pd.DataFrame(result,index = [0])
        return result_df
    df = df.sort_values(by ='salary')
    value = df.salary.iloc[N-1]
    result ={f' getNthHighestSalary({N})':value}
    result_df = pd.DataFrame(result,index = [0])
    return result_df

# second problem
def second_highest_salary(employee:pd.DataFrame)->pd.DataFrame:
    employee = employee.drop_duplicates(subset=['salary'])
    if len(employee)<2:
        dic = {'SecondHighestSalary':np.NaN}
        df = pd.DataFrame(dic,index=[0])
        return df
    employee = employee.sort_values(by='salary',ascending= False)
    second_highest = employee.salary.iloc[1]
    dic = {'SecondHighestSalary':second_highest}
    df = pd.DataFrame(dic,index=[0])
    return df

# third problem
def hieghst_salary_dep(employee:pd.DataFrame, departemnt:pd.DataFrame)-> pd.DataFrame:
    merged_df = pd.merge(employee,departemnt,how='left',left_on='departmentId',right_on ='id')
    merged_df = merged_df[['name_y', 'name_x', 'salary']].rename(columns={'name_y': 'Department', 'name_x': 'Employee', 'salary': 'Salary'})
    merged_df = merged_df.sort_values(by='Department', ascending=True)
    # I didn't solve the lambda myself 
    merged_df = merged_df.groupby('Department').apply(lambda group : group[group['Salary']==group['Salary'].max()])
    merged_df = merged_df.reset_index(drop=True)
    return merged_df
    
# fourth problem 
# my intial solution put it didn't pass the last test case
def order_scores(scores: pd.DataFrame) -> pd.DataFrame:
    if len(scores)==0:
        return None
    scores = scores.sort_values(by='score', ascending=False).reset_index(drop=True)
    rank = 1
    scores.at[0, 'rank'] = 1
    for i in range(1, len(scores)):
        if scores['score'][i] != scores['score'][i - 1]:
            rank += 1
        scores.at[i, 'rank'] = rank
    return scores[['score','rank']]

#fifth problem
def delete_duplicate_emails(person: pd.DataFrame) -> None:
    person.sort_values(by='id',ascending=True,inplace=True)
    person.drop_duplicates(subset='email',keep='first',inplace=True)

#sixth problem
def rearrange_products_table(products: pd.DataFrame) -> pd.DataFrame:
    melted_df = pd.melt(products,id_vars=['product_id'],value_name='price',var_name='store')
    return melted_df.dropna()


### Statistics

In [5]:
# third problem
# initial solution
def count_salary_categories(accounts: pd.DataFrame) -> pd.DataFrame:
    dic = {'High Salary':0,'Low Salary':0,'Average Salary':0}
    for i in range(len(accounts['income'])):
        if accounts['income'][i] < 20000:
            dic['Low Salary']+=1
        elif 20000 <= accounts['income'][i]<= 50000:
            dic['Average Salary']+=1
        else :
            dic['High Salary']+=1
    
    df = pd.DataFrame()
    df['category'] = dic.keys()
    df['accounts_count'] = dic.values()
    return df

# better solution

def count_salary_categories(accounts: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame({
        'category':['High Salary','Low Salary','Average Salary'],
        'accounts_count':[
            accounts[accounts['income']>50000].shape[0],
            accounts[accounts['income']<20000].shape[0],
            accounts[(accounts['income']>=20000)&(accounts['income']<=50000)].shape[0]
        ]
    })

### Data Aggregation


In [6]:
# first problem 
def total_time(employees: pd.DataFrame) -> pd.DataFrame:
    return employees.assign(
    total_time = employees['out_time'] - employees['in_time'],
    day = employee['event_day'].dt.strftime('%Y-%m-%d')
    ).groupby(['emp_id','day'],as_index=False)['total_time'].sum()[['day','emp_id','total_time']]
    
# second problem
#initial solution 
def game_analysis(activity:pd.DataFrame)-> pd.DataFrame:
    return activity.groupby('player_id').apply(lambda group : group[group['event_date'] == group['event_date'].min()]).\
    rename(columns={'event_date':'first_login'})[['player_id','first_login']].reset_index(drop=True)

# third problem 
def count_unique_subjects(teacher: pd.DataFrame) -> pd.DataFrame:
    return teacher.groupby(
        'teacher_id',as_index=False)['subject_id'].nunique().rename(columns={'subject_id':'cnt'})

#fourth problem 
# initial solution
def find_classes(courses: pd.DataFrame) -> pd.DataFrame:
    group =  courses.groupby('class')['class'].count()
    valid = group.loc[group>=5].index
    return pd.DataFrame(valid,columns=['class'])
# better solutin
def find_classes(courses: pd.DataFrame) -> pd.DataFrame:
    groups = courses.groupby('class',as_index=False)['student'].count()
    return groups[groups['student']>=5][['class']]

# fifth problem 
def largest_orders(orders: pd.DataFrame) -> pd.DataFrame:
    groups = orders.groupby('customer_number',as_index = False).agg(order_count =('order_number','count'))
    return groups[groups['order_count']==groups['order_count'].max()][['customer_number']].reset_index(drop = True)

# sexith problem 
def categorize_products(activities: pd.DataFrame) -> pd.DataFrame:
    return activities.groupby(
        'sell_date'
    )['product'].agg([
        ('num_sold', 'nunique'),
        ('products', lambda x: ','.join(sorted(x.unique())))
    ]).reset_index()

# seventh problem
def daily_leads_and_partners(daily_sales: pd.DataFrame) -> pd.DataFrame:
    return daily_sales.groupby(['date_id','make_name'],as_index = False).\
            agg(unique_leads = ('lead_id','nunique'),
               unique_partners = ('partner_id','nunique'))

    


## Data Integration


In [7]:
# first problem 
def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    grouped = actor_director.groupby(['actor_id','director_id'],as_index=False).count()
    return grouped[grouped['timestamp']>=3][['actor_id','director_id']]

# second problem
# initial solution 
def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    return pd.merge(
        employees, employee_uni, how='left', on='id'
    )[['unique_id', 'name']]

# third problem
def students_and_examinations(students: pd.DataFrame, subjects: pd.DataFrame, examinations: pd.DataFrame) -> pd.DataFrame:
    # create combinaiton for all subjects for each student
    all_combination = pd.merge(students.assign(key = 1),subjects.assign(key = 1),on='key',how ='left').drop('key',axis=1)
    # now we calculate the number of subjects each student attended
    attended_exmas = examinations.groupby(['student_id','subject_name'], as_index = False).agg(
                                                    attended_exams = ('subject_name','count')
                                                    )
    # now to add the cases that has no values for attending the exams we group the togheter
    # we add the second df by merging as left to the first one so that it add nan for unavilable combinaiton
    final = pd.merge(all_combination,attended_exmas,on =['student_id','subject_name'],how = 'left').fillna(0)
    return final[['student_id', 'student_name', 'subject_name', 'attended_exams']].sort_values(by=['student_id','subject_name'])

## approved solution
def students_and_examinations(students: pd.DataFrame, subjects: pd.DataFrame, examinations: pd.DataFrame) -> pd.DataFrame:
    # group by student_id', 'subject_name'; getting count for each subject
    examinations = examinations.groupby(['student_id', 'subject_name']).agg(attended_exams=('subject_name', 'count')).reset_index() 
    # cross join
    students = students.merge(subjects, how='cross')
    # right join
    examinations = examinations.merge(students, on=['student_id', 'subject_name'],how='right')
    # filling null values with 0
    examinations = examinations.fillna({'attended_exams': 0})
    # sorting by 'student_id', 'subject_name'
    examinations = examinations.sort_values(['student_id', 'subject_name'])
    return examinations[['student_id', 'student_name', 'subject_name', 'attended_exams']]

# fourth problem
# initial solution didn't pass the 7th test case
def find_managers(employee: pd.DataFrame) -> pd.DataFrame:
    # counting the reports count for each manager 
    manager_group = employee.groupby('managerId').agg(reports_count = ('managerId','count'))
    # merging the counted reports to original df
    grouped_df = pd.merge(employee, manager_group, left_on='id',right_on ='managerId',how = 'left')
    return grouped_df[(grouped_df['managerId'].isna())&(grouped_df['reports_count']>=5)][['name']]

# approved solution
def find_managers(employee:pd.DataFrame)->pd.DataFrame:
    # first find the id of the managers
    managers = employee.groupby('managerId',as_index = False).agg(
                                                        reports_count = ('managerId','count')
                                                            ).query(
                                                        '5<= reports_count'
                                                            )['managerId']
    return employee[employee['id'].isin(managers)][['name']]

# final problem
def sales_person(sales_person: pd.DataFrame, company: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    return sales_person[
        ~sales_person['sales_id'].isin(
            pd.merge(
                left=orders,
                right=company[company['name'] == 'RED'],
                on='com_id',
                how='inner',
            )['sales_id'].unique()
        )
    ][['name']]